In [48]:
import cv2
import numpy as np
import os

# Load video and create a tracker object
video = cv2.VideoCapture(r"C:\Users\pc_al\Desktop\New folder\sighn_V1.mp4")
save_path=r"C:\Users\pc_al\Desktop\New folder"
class_=0

width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)


#if you wnat to change track algorithm
tracker = cv2.TrackerCSRT_create()

labels=[]
Frames=[]
# Read first frame and define region of interest (ROI)
ret, frame = video.read()


bbox = cv2.selectROI(frame, True)
tracker.init(frame, bbox)

Frames.append(frame.copy())
labels.append(list(bbox))

checker=[frame.copy()]

# Loop through video frames and track object
while True:
    ret, frame = video.read()
    if not ret:
        break

    checker.append(frame.copy())
        
    #To see if there is a difference between the current frame and the previous one
    diff=cv2.absdiff(checker[1], checker[0])

    
    mean = np.mean(diff)
    std = np.std(diff)
            
    print("mean of The difference between the two frames :",mean)
    print("std of The difference between the two frames :",std)
    print('=======')
            
    #Determine the value that determines if there is a difference or not
    if (mean >15) and (std > 15):
        #if you wnat to change track algorithm
        tracker = cv2.TrackerCSRT_create()
        bbox = cv2.selectROI(frame, True)
        
        Frames.append(frame.copy())
        labels.append(list(bbox))
        
        tracker.init(frame, bbox)
        
        ret, frame = video.read()
        if not ret:
            break
        
         
    # Update tracker and get new bounding box coordinates
    success, bbox = tracker.update(frame)
    if success:
        Frames.append(frame.copy())
        labels.append(list(bbox))
        # Draw bounding box on frame
        (x, y, w, h) = [int(i) for i in bbox]
        cv2.rectangle(frame, (x, y), (x + w,y+h), (0, 255, 0), 2)
        cv2.imshow('Frame', frame)
    else:
        cv2.putText(frame, "Tracking failed", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        # Display frame
        cv2.imshow('Frame', frame)
                        
                        
    del checker[0]
                        
    # Exit loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video and close window
video.release()
cv2.destroyAllWindows()



mean of The difference between the two frames : 2.033741666666667
std of The difference between the two frames : 7.0446755189005845
mean of The difference between the two frames : 1.2362583333333332
std of The difference between the two frames : 2.857281400200294
mean of The difference between the two frames : 1.7125416666666666
std of The difference between the two frames : 5.00918583270747
mean of The difference between the two frames : 2.2616916666666667
std of The difference between the two frames : 7.807791566010448
mean of The difference between the two frames : 2.47455
std of The difference between the two frames : 8.707019331024442
mean of The difference between the two frames : 2.354225
std of The difference between the two frames : 8.339202578746663
mean of The difference between the two frames : 2.1919416666666667
std of The difference between the two frames : 7.825457392165473
mean of The difference between the two frames : 2.1982916666666665
std of The difference between t

In [49]:
TRYlabel=labels.copy()

for i in range(len(TRYlabel)):
    for j in range(len(TRYlabel[i])):
        if TRYlabel[i][j] <0:
            TRYlabel[i][j]=0
            
        elif TRYlabel[i][j]>200:
            TRYlabel[i][j]=200
            
normalized_label=[]
for i in TRYlabel:
    x,y,w,h=i
    
    xmin=x/width
    ymin=y/height
    
    xmax=(x+w)/width
    ymax=(y+h)/height
    
    normalized_label.append([class_,xmin,ymin,xmax,ymax])





In [50]:

if not os.path.exists("data"):
    # Create new folder
    os.mkdir(f"{save_path}\data")
    os.mkdir(f"{save_path}\data\image")
    os.mkdir(f"{save_path}\data\label")
else:
    print("Folder already exists.")



for i in range(len(normalized_label)):

    cv2.imwrite(f"{save_path}\data\image\({i}).jpg", Frames[i])
    
    file = open(f"{save_path}\data\label\({i}).txt", "w")
    file.write(' '.join(map(str, normalized_label[i])))
    file.close()
